In [5]:
!pip install gdown
!gdown --id 1IQ90jtnITrrcBWsFjF8jkFXF7LAxDqLF


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\temic\AppData\Local\Programs\Python\Python310\lib\site-packages\gdown\__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1IQ90jtnITrrcBWsFjF8jkFXF7LAxDqLF

but Gdown can't. Please check connections and permissions.


In [2]:
import matplotlib.pyplot as plt
def show_images(images, figsize=(20,10), columns = 5):
    plt.figure(figsize=figsize)
    for i, image in enumerate(images):
        plt.subplot(len(images) // columns + 1, columns, i + 1)
        plt.imshow(image)
        

In [3]:
import os
IMAGES_PATH="./images"
file_names=os.listdir(IMAGES_PATH)
print(f"number of images: {len(file_names)}")


number of images: 5762


In [4]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
import os
from os import listdir
from os.path import splitext
import numpy as np
from PIL import Image
import pickle as pk
from tqdm import tqdm
def read_img_file(f):
    img = Image.open(f)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img

def resize_img_to_array(img, img_shape):
    img_array = np.array(
        img.resize(
            img_shape,
            Image.ANTIALIAS
        )
    )
    return img_array

def get_features(img):
    img_width, img_height = 224, 224
    np_img = resize_img_to_array(img, img_shape=(img_width, img_height))
    expanded_img_array = np.expand_dims(np_img, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    X_conv = model.predict(preprocessed_img)
    image_features=X_conv[0]
    image_features /=  np.linalg.norm(image_features)
    return image_features

model = ResNet50(weights='imagenet', include_top=False,input_shape=(224, 224, 3),pooling='max')

def generate_resnet_features():
    all_image_features=[]
    image_filenames=listdir(IMAGES_PATH)
    image_ids=set(map(lambda el: splitext(el)[0],image_filenames))
    try:
        all_image_features=pk.load(open("resnet_image_features.pkl", "rb"))
    except (OSError, IOError) as e:
        print("file_not_found")

    def exists_in_all_image_features(image_id):
        for image in all_image_features:
            if image['image_id'] == image_id:
                # print("skipping "+ str(image_id))
                return True
        return False

    def exists_in_image_folder(image_id):
        if image_id in image_ids:
            return True
        return False

    def sync_resnet_image_features():
        for_deletion=[]
        for i in range(len(all_image_features)):
            if not exists_in_image_folder(all_image_features[i]['image_id']):
                print("deleting "+ str(all_image_features[i]['image_id']))
                for_deletion.append(i)
        for i in reversed(for_deletion):
            del all_image_features[i]

    sync_resnet_image_features()
    for image_filename in tqdm(image_filenames):
        image_id=splitext(image_filename)[0]
        if exists_in_all_image_features(image_id):
            continue
        img_arr = read_img_file(IMAGES_PATH+"/"+image_filename)
        image_features=get_features(img_arr)
        # print(image_filename)
        # print(image_features)
        all_image_features.append({'image_id':image_id,'features':image_features})
    pk.dump(all_image_features, open("resnet_image_features.pkl","wb"))

In [5]:
%%capture
generate_resnet_features()

MemoryError: 

In [1]:
import numpy as np
from PIL import Image
query_image_pillow=Image.open(f'{IMAGES_PATH}/0a3e62e3-fac5-4648-9da2-f6bc4074ee31.jpg').convert('RGB')
query_image_features=get_features(query_image_pillow)
show_images([np.array(query_image_pillow)])
print(query_image_features.shape)

NameError: name 'IMAGES_PATH' is not defined

In [ ]:
from sklearn.neighbors import NearestNeighbors
from os import listdir
import numpy as np
import pickle as pk
import json
from pathlib import Path

image_features=pk.load( open("resnet_image_features.pkl", "rb"))
features=[]
for image in image_features:
    features.append(np.array(image['features']))
features=np.array(features)
features=np.squeeze(features)

path="./images"
knn = NearestNeighbors(n_neighbors=20,algorithm='brute',metric='euclidean')
knn.fit(features)
file_names=listdir(path)

indices = knn.kneighbors([query_image_features], return_distance=False)
found_images=[]
for x in indices[0]:
    found_images.append(np.array(Image.open(path+"/"+file_names[x])))
show_images(np.array(found_images))